## Dataset Download

In [ ]:
!gdown 1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU

Downloading...
From (original): https://drive.google.com/uc?id=1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU
From (redirected): https://drive.google.com/uc?id=1doU9FE1tJ-0IL4tKZUOOp76JkXKuTkFU&confirm=t&uuid=52071f04-cc2f-414e-b37c-ce4972b9c905
To: /kaggle/working/anxiety_dataset_complete.zip
100%|███████████████████████████████████████| 1.41G/1.41G [00:04<00:00, 321MB/s]


In [ ]:
!unzip anxiety_dataset_complete.zip > /dev/null

In [27]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import math

TRAIN_RATIO = 0.7  # 70% train
VAL_RATIO = 0.1    # 10% validation
TEST_RATIO = 0.2   # 20% test
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
LR = 2e-5
MAX_LEN = 512
FUSION_DIM = 768
LABELS = ["Nervousness", "Lack of Worry Control", "Excessive Worry",
          "Difficulty Relaxing", "Restlessness", "Impending Doom"]
LABEL_MAP = {i: idx for idx, i in enumerate(LABELS)}

## Custom Dataset

In [28]:
class TextOnlyAnxietyDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        ocr_text = sample["ocr_text"]
        figurative_reasoning = sample["figurative_reasoning"]
        combined_text = ocr_text + " [SEP] " + figurative_reasoning

        encoding = self.tokenizer(
            combined_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )

        label = LABEL_MAP[sample["meme_anxiety_category"]]

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

def custom_collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'label': labels,
    }

## Model Definition

In [29]:
class TextOnlyModel(nn.Module):
    def __init__(self, text_model_name="bert-base-uncased", num_classes=6, hidden_dim=768):
        super(TextOnlyModel, self).__init__()

        self.text_encoder = AutoModel.from_pretrained(text_model_name)
        self.text_dim = self.text_encoder.config.hidden_size

        self.text_projections = nn.ModuleList([
            nn.Linear(self.text_dim, hidden_dim) for _ in range(3)
        ])

        self.expert_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.GELU(),
                nn.Dropout(0.2)
            ) for _ in range(4)
        ])

        self.moe_gate = nn.Linear(hidden_dim, 4)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

        text_features = [
            text_outputs.hidden_states[-i][:, 0] for i in range(1, 4)
        ]
        text_features = [
            proj(feat) for proj, feat in zip(self.text_projections, text_features)
        ]

        combined_feature = sum(text_features) / len(text_features)

        expert_outputs = [expert(combined_feature) for expert in self.expert_nets]
        expert_gates = F.softmax(self.moe_gate(combined_feature), dim=1)

        moe_output = torch.zeros_like(expert_outputs[0])
        for i, expert_out in enumerate(expert_outputs):
            moe_output += expert_out * expert_gates[:, i].unsqueeze(1)

        logits = self.classifier(moe_output)

        return logits

## Training Functions

In [30]:
def train_text_model(model, train_data, val_data, epochs, model_save_name):
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

    train_dataset = TextOnlyAnxietyDataset(train_data, tokenizer, max_len=MAX_LEN)
    val_dataset = TextOnlyAnxietyDataset(val_data, tokenizer, max_len=MAX_LEN)

    print("Train Set Size:", len(train_dataset))
    print("Validation Set Size:", len(val_dataset))
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=custom_collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    criterion = nn.CrossEntropyLoss()

    model = model.to(DEVICE)
    model = nn.DataParallel(model)

    best_f1 = 0
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        model.train()
        train_loss = 0
        train_preds, train_labels = [], []

        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            train_preds.extend(predictions)
            train_labels.extend(labels.cpu().numpy())

        train_loss = train_loss / len(train_loader)
        train_macro_f1 = f1_score(train_labels, train_preds, average="macro")
        train_weighted_f1 = f1_score(train_labels, train_preds, average="weighted")

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Macro-F1: {train_macro_f1:.4f}, Weighted-F1: {train_weighted_f1:.4f}")

        val_loss, val_macro_f1, val_weighted_f1 = evaluate_text_model(
            model, val_loader, criterion
        )

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro-F1: {val_macro_f1:.4f}, Weighted-F1: {val_weighted_f1:.4f}")

        scheduler.step(val_macro_f1)

        f1_hm = 2 * val_macro_f1 * val_weighted_f1 / (val_macro_f1 + val_weighted_f1)
        if f1_hm > best_f1:
            best_f1 = f1_hm
            torch.save(model.state_dict(), f"{model_save_name}_anxiety.pth")
            print("Best model saved!")

    return model

def evaluate_text_model(model, loader, criterion):
    model.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            val_preds.extend(predictions)
            val_labels.extend(labels.cpu().numpy())

    val_loss = val_loss / len(loader)
    val_macro_f1 = f1_score(val_labels, val_preds, average="macro")
    val_weighted_f1 = f1_score(val_labels, val_preds, average="weighted")

    target_names = [k for k, v in sorted(LABEL_MAP.items(), key=lambda x: x[1])]
    report = classification_report(val_labels, val_preds, target_names=target_names)
    print(report)

    return val_loss, val_macro_f1, val_weighted_f1

## Model Training

In [ ]:
full_train_data = json.load(open("anxiety_train_llava_dataset.json", "r"))
test_data = json.load(open("anxiety_test_llava_dataset.json", "r"))

labels = [LABEL_MAP[item["meme_anxiety_category"]] for item in full_train_data]

train_size = math.ceil(len(full_train_data) * TRAIN_RATIO / (TRAIN_RATIO + VAL_RATIO))
train_data, val_data = train_test_split(
    full_train_data, train_size=train_size, stratify=labels, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

test_dataset = TextOnlyAnxietyDataset(test_data, tokenizer, max_len=MAX_LEN)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=custom_collate_fn
)

In [ ]:
model = TextOnlyModel(
    text_model_name="mental/mental-roberta-base",
    num_classes=len(LABEL_MAP),
    hidden_dim=FUSION_DIM
)

trained_model, tokenizer = train_text_model(
    model,
    train_data,
    val_data,
    epochs=30,
    model_save_name="only_text"
)

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Set Size: 2153
Validation Set Size: 307

Epoch 1/30


Training: 100%|██████████| 135/135 [02:01<00:00,  1.11it/s]


Train Loss: 1.7209
Train Macro-F1: 0.2361, Weighted-F1: 0.2382


Evaluating: 100%|██████████| 20/20 [00:05<00:00,  3.70it/s]


...

                       precision    recall  f1-score   support

          Nervousness       0.65      0.70      0.67        53
Lack of Worry Control       0.69      0.70      0.69        47
      Excessive Worry       0.62      0.50      0.55        46
  Difficulty Relaxing       0.90      0.86      0.88        51
         Restlessness       0.65      0.67      0.66        58
       Impending Doom       0.68      0.73      0.70        52

             accuracy                           0.70       307
            macro avg       0.70      0.69      0.69       307
         weighted avg       0.70      0.70      0.70       307

Validation Loss: 2.3026
Validation Macro-F1: 0.6944, Weighted-F1: 0.6958

Epoch 30/30


Training: 100%|██████████| 135/135 [02:01<00:00,  1.11it/s]


Train Loss: 0.0059
Train Macro-F1: 0.9976, Weighted-F1: 0.9977


Evaluating: 100%|██████████| 20/20 [00:05<00:00,  3.71it/s]


                       precision    recall  f1-score   support

          Nervousness       0.65      0.70      0.67        53
Lack of Worry Control       0.69      0.70      0.69        47
      Excessive Worry       0.62      0.50      0.55        46
  Difficulty Relaxing       0.90      0.86      0.88        51
         Restlessness       0.65      0.67      0.66        58
       Impending Doom       0.68      0.73      0.70        52

             accuracy                           0.70       307
            macro avg       0.70      0.69      0.69       307
         weighted avg       0.70      0.70      0.70       307

Validation Loss: 2.2974
Validation Macro-F1: 0.6944, Weighted-F1: 0.6958


## Inference

In [32]:
!gdown 1nYT2Dw1vUTh5iK7FNSLwlJq1921_FjTO

Downloading...
From (original): https://drive.google.com/uc?id=1nYT2Dw1vUTh5iK7FNSLwlJq1921_FjTO
From (redirected): https://drive.google.com/uc?id=1nYT2Dw1vUTh5iK7FNSLwlJq1921_FjTO&confirm=t&uuid=3f0e6ee0-c984-43dc-aa27-4d8cb5bb78a3
To: /kaggle/working/only_text_anxiety.pth
100%|████████████████████████████████████████| 518M/518M [00:12<00:00, 42.1MB/s]


In [38]:
def inference(test_data, model_path):
    model = TextOnlyModel(
        text_model_name="mental/mental-roberta-base",
        num_classes=len(LABEL_MAP),
        hidden_dim=FUSION_DIM
    )

    model = model.to(DEVICE)

    weights = torch.load(model_path, map_location=DEVICE, weights_only=True)
    weights_single = {k.replace("module.", ""): v for k, v in weights.items()}

    model.load_state_dict(weights_single)
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")

    test_dataset = TextOnlyAnxietyDataset(test_data, tokenizer, max_len=MAX_LEN)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=custom_collate_fn
    )

    loss, macro_f1, weighted_f1 = evaluate_text_model(
        model, test_loader, nn.CrossEntropyLoss()
    )

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Macro-F1: {macro_f1:.4f}, Weighted-F1: {weighted_f1:.4f}")

    return macro_f1, weighted_f1

In [39]:
inference(test_data, "only_text_anxiety.pth")

Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 39/39 [00:20<00:00,  1.93it/s]

                       precision    recall  f1-score   support

          Nervousness       0.70      0.60      0.65       106
Lack of Worry Control       0.65      0.67      0.66        94
      Excessive Worry       0.62      0.60      0.61        92
  Difficulty Relaxing       0.79      0.76      0.78       102
         Restlessness       0.50      0.59      0.54       116
       Impending Doom       0.74      0.72      0.73       105

             accuracy                           0.66       615
            macro avg       0.67      0.66      0.66       615
         weighted avg       0.66      0.66      0.66       615

Test Loss: 2.5080
Test Macro-F1: 0.6606, Weighted-F1: 0.6590


(0.6606387991478248, 0.658952957544853)